In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px
from profolio import Profolio

profolio = Profolio('Firstrade')
profolio.from_csv('data/FT_CSV_87701987.csv')

In [3]:
from chart import Chart
new_chart = Chart(profolio)
new_chart.ROE_DayHeld()
new_chart.WinRate_GainLose()